### RAG Pipelines- Data Ingestion to Vector DB Pipeline

In [3]:
!pip install langchain_community
!pip install langchain-text-splitters

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 70.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.1/489.1 kB 49.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 3.4 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 1.2.1
    Uninstalling langchain-core-1.2.1:
      Successfully uninstalled langchain-core-1.2.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is inc

In [4]:
import os
from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from pathlib import Path

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.6/329.6 kB 6.6 MB/s eta 0:00:00


In [7]:
### Read all the pdf's inside the directory
def process_all_pdfs(pdf_directory):
    """Process all PDF files in a directory"""
    all_documents = []
    pdf_dir = Path(pdf_directory)

    # Find all PDF files recursively
    pdf_files = list(pdf_dir.glob("**/*.pdf"))

    print(f"Found {len(pdf_files)} PDF files to process")

    for pdf_file in pdf_files:
        print(f"\nProcessing: {pdf_file.name}")
        try:
            loader = PyPDFLoader(str(pdf_file))
            documents = loader.load()

            # Add source information to metadata
            for doc in documents:
                doc.metadata['source_file'] = pdf_file.name
                doc.metadata['file_type'] = 'pdf'

            all_documents.extend(documents)
            print(f"  ✓ Loaded {len(documents)} pages")

        except Exception as e:
            print(f"  ✗ Error: {e}")

    print(f"\nTotal documents loaded: {len(all_documents)}")
    return all_documents

# Process all PDFs in the data directory
all_pdf_documents = process_all_pdfs("/content/drive/MyDrive/RAG/rag_pipline_project_1/data/pdf")

Found 1 PDF files to process

Processing: Thermal-imaging-cameras-testing-solar-panels.pdf
  ✓ Loaded 4 pages

Total documents loaded: 4


In [8]:
all_pdf_documents

[Document(metadata={'producer': 'Adobe PDF Library 9.9', 'creator': 'Adobe InDesign CS5 (7.0)', 'creationdate': '2011-03-08T19:02:03+08:00', 'moddate': '2011-03-08T19:02:05+08:00', 'trapped': '/False', 'source': '/content/drive/MyDrive/RAG/rag_pipline_project_1/data/pdf/Thermal-imaging-cameras-testing-solar-panels.pdf', 'total_pages': 4, 'page': 0, 'page_label': '1', 'source_file': 'Thermal-imaging-cameras-testing-solar-panels.pdf', 'file_type': 'pdf'}, page_content="Technical no Te\nwww.flir.com\nIn the field of research and development \n(R&D) thermal imaging cameras are already \nan established tool for the evaluation of \nsolar cells and panels. For these sophisticated \nmeasurements, usually high performance \ncameras with cooled detectors are used \nunder controlled laboratory conditions.\nHowever, the use of thermal imaging cameras \nfor solar panel evaluation is not restricted \nto the field of research. Uncooled thermal \nimaging cameras are currently being used \nmore and mor

In [9]:
### Text splitting get into chunks

def split_documents(documents,chunk_size=1000,chunk_overlap=200):
    """Split documents into smaller chunks for better RAG performance"""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n\n", "\n", " ", ""]
    )
    split_docs = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(split_docs)} chunks")

    # Show example of a chunk
    if split_docs:
        print(f"\nExample chunk:")
        print(f"Content: {split_docs[0].page_content[:200]}...")
        print(f"Metadata: {split_docs[0].metadata}")

    return split_docs


In [10]:
chunks=split_documents(all_pdf_documents)
chunks

Split 4 documents into 19 chunks

Example chunk:
Content: Technical no Te
www.flir.com
In the field of research and development 
(R&D) thermal imaging cameras are already 
an established tool for the evaluation of 
solar cells and panels. For these sophistic...
Metadata: {'producer': 'Adobe PDF Library 9.9', 'creator': 'Adobe InDesign CS5 (7.0)', 'creationdate': '2011-03-08T19:02:03+08:00', 'moddate': '2011-03-08T19:02:05+08:00', 'trapped': '/False', 'source': '/content/drive/MyDrive/RAG/rag_pipline_project_1/data/pdf/Thermal-imaging-cameras-testing-solar-panels.pdf', 'total_pages': 4, 'page': 0, 'page_label': '1', 'source_file': 'Thermal-imaging-cameras-testing-solar-panels.pdf', 'file_type': 'pdf'}


[Document(metadata={'producer': 'Adobe PDF Library 9.9', 'creator': 'Adobe InDesign CS5 (7.0)', 'creationdate': '2011-03-08T19:02:03+08:00', 'moddate': '2011-03-08T19:02:05+08:00', 'trapped': '/False', 'source': '/content/drive/MyDrive/RAG/rag_pipline_project_1/data/pdf/Thermal-imaging-cameras-testing-solar-panels.pdf', 'total_pages': 4, 'page': 0, 'page_label': '1', 'source_file': 'Thermal-imaging-cameras-testing-solar-panels.pdf', 'file_type': 'pdf'}, page_content='Technical no Te\nwww.flir.com\nIn the field of research and development \n(R&D) thermal imaging cameras are already \nan established tool for the evaluation of \nsolar cells and panels. For these sophisticated \nmeasurements, usually high performance \ncameras with cooled detectors are used \nunder controlled laboratory conditions.\nHowever, the use of thermal imaging cameras \nfor solar panel evaluation is not restricted \nto the field of research. Uncooled thermal \nimaging cameras are currently being used \nmore and mor

### Embedding And VectorStoreDB

In [11]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.7/21.7 MB 95.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 103.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 128.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.0/220.0 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 4.0 MB/s et

In [12]:
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid
from typing import List, Dict, Any, Tuple
from sklearn.metrics.pairwise import cosine_similarity

In [13]:
class EmbeddingManager:
    """Handles document embedding generation using SentenceTransformer"""

    def __init__(self, model_name: str = "all-MiniLM-L6-v2"):
        """
        Initialize the embedding manager

        Args:
            model_name: HuggingFace model name for sentence embeddings
        """
        self.model_name = model_name
        self.model = None
        self._load_model()

    def _load_model(self):
        """Load the SentenceTransformer model"""
        try:
            print(f"Loading embedding model: {self.model_name}")
            self.model = SentenceTransformer(self.model_name)
            print(f"Model loaded successfully. Embedding dimension: {self.model.get_sentence_embedding_dimension()}")
        except Exception as e:
            print(f"Error loading model {self.model_name}: {e}")
            raise

    def generate_embeddings(self, texts: List[str]) -> np.ndarray:
        """
        Generate embeddings for a list of texts

        Args:
            texts: List of text strings to embed

        Returns:
            numpy array of embeddings with shape (len(texts), embedding_dim)
        """
        if not self.model:
            raise ValueError("Model not loaded")

        print(f"Generating embeddings for {len(texts)} texts...")
        embeddings = self.model.encode(texts, show_progress_bar=True)
        print(f"Generated embeddings with shape: {embeddings.shape}")
        return embeddings


## initialize the embedding manager

embedding_manager=EmbeddingManager()
embedding_manager


Loading embedding model: all-MiniLM-L6-v2


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Model loaded successfully. Embedding dimension: 384


### VectorStore

In [14]:
import os

class VectorStore:
    """Manages document embeddings in a ChromaDB vector store"""

    def __init__(self, collection_name: str = "pdf_documents", persist_directory: str = "/content/drive/MyDrive/RAG/rag_pipline_project_1/data/vector_store"):
        """
        Initialize the vector store

        Args:
            collection_name: Name of the ChromaDB collection
            persist_directory: Directory to persist the vector store
        """
        self.collection_name = collection_name
        self.persist_directory = persist_directory
        self.client = None
        self.collection = None
        self._initialize_store()

    def _initialize_store(self):
        """Initialize ChromaDB client and collection"""
        try:
            # Create persistent ChromaDB client
            os.makedirs(self.persist_directory, exist_ok=True)
            self.client = chromadb.PersistentClient(path=self.persist_directory)

            # Get or create collection
            self.collection = self.client.get_or_create_collection(
                name=self.collection_name,
                metadata={"description": "PDF document embeddings for RAG"}
            )
            print(f"Vector store initialized. Collection: {self.collection_name}")
            print(f"Existing documents in collection: {self.collection.count()}")

        except Exception as e:
            print(f"Error initializing vector store: {e}")
            raise

    def add_documents(self, documents: List[Any], embeddings: np.ndarray):
        """
        Add documents and their embeddings to the vector store

        Args:
            documents: List of LangChain documents
            embeddings: Corresponding embeddings for the documents
        """
        if len(documents) != len(embeddings):
            raise ValueError("Number of documents must match number of embeddings")

        print(f"Adding {len(documents)} documents to vector store...")

        # Prepare data for ChromaDB
        ids = []
        metadatas = []
        documents_text = []
        embeddings_list = []

        for i, (doc, embedding) in enumerate(zip(documents, embeddings)):
            # Generate unique ID
            doc_id = f"doc_{uuid.uuid4().hex[:8]}_{i}"
            ids.append(doc_id)

            # Prepare metadata
            metadata = dict(doc.metadata)
            metadata['doc_index'] = i
            metadata['content_length'] = len(doc.page_content)
            metadatas.append(metadata)

            # Document content
            documents_text.append(doc.page_content)

            # Embedding
            embeddings_list.append(embedding.tolist())

        # Add to collection
        try:
            self.collection.add(
                ids=ids,
                embeddings=embeddings_list,
                metadatas=metadatas,
                documents=documents_text
            )
            print(f"Successfully added {len(documents)} documents to vector store")
            print(f"Total documents in collection: {self.collection.count()}")

        except Exception as e:
            print(f"Error adding documents to vector store: {e}")
            raise

vectorstore=VectorStore()
vectorstore


Vector store initialized. Collection: pdf_documents
Existing documents in collection: 76


In [15]:
chunks

[Document(metadata={'producer': 'Adobe PDF Library 9.9', 'creator': 'Adobe InDesign CS5 (7.0)', 'creationdate': '2011-03-08T19:02:03+08:00', 'moddate': '2011-03-08T19:02:05+08:00', 'trapped': '/False', 'source': '/content/drive/MyDrive/RAG/rag_pipline_project_1/data/pdf/Thermal-imaging-cameras-testing-solar-panels.pdf', 'total_pages': 4, 'page': 0, 'page_label': '1', 'source_file': 'Thermal-imaging-cameras-testing-solar-panels.pdf', 'file_type': 'pdf'}, page_content='Technical no Te\nwww.flir.com\nIn the field of research and development \n(R&D) thermal imaging cameras are already \nan established tool for the evaluation of \nsolar cells and panels. For these sophisticated \nmeasurements, usually high performance \ncameras with cooled detectors are used \nunder controlled laboratory conditions.\nHowever, the use of thermal imaging cameras \nfor solar panel evaluation is not restricted \nto the field of research. Uncooled thermal \nimaging cameras are currently being used \nmore and mor

In [16]:
### Convert the text to embeddings
texts=[doc.page_content for doc in chunks]

## Generate the Embeddings
embeddings=embedding_manager.generate_embeddings(texts)

##Store in the Vector Database
vectorstore.add_documents(chunks,embeddings)

Generating embeddings for 19 texts...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Generated embeddings with shape: (19, 384)
Adding 19 documents to vector store...
Successfully added 19 documents to vector store
Total documents in collection: 95


## Retriever Pipeline From VectorStore

In [17]:
class RAGRetriever:
    """Handles query-based retrieval from the vector store"""

    def __init__(self, vector_store: VectorStore, embedding_manager: EmbeddingManager):
        """
        Initialize the retriever

        Args:
            vector_store: Vector store containing document embeddings
            embedding_manager: Manager for generating query embeddings
        """
        self.vector_store = vector_store
        self.embedding_manager = embedding_manager

    def retrieve(self, query: str, top_k: int = 5, score_threshold: float = 0.0) -> List[Dict[str, Any]]:
        """
        Retrieve relevant documents for a query

        Args:
            query: The search query
            top_k: Number of top results to return
            score_threshold: Minimum similarity score threshold

        Returns:
            List of dictionaries containing retrieved documents and metadata
        """
        print(f"Retrieving documents for query: '{query}'")
        print(f"Top K: {top_k}, Score threshold: {score_threshold}")

        # Generate query embedding
        query_embedding = self.embedding_manager.generate_embeddings([query])[0]

        # Search in vector store
        try:
            results = self.vector_store.collection.query(
                query_embeddings=[query_embedding.tolist()],
                n_results=top_k
            )

            # Process results
            retrieved_docs = []

            if results['documents'] and results['documents'][0]:
                documents = results['documents'][0]
                metadatas = results['metadatas'][0]
                distances = results['distances'][0]
                ids = results['ids'][0]

                for i, (doc_id, document, metadata, distance) in enumerate(zip(ids, documents, metadatas, distances)):
                    # Convert distance to similarity score (ChromaDB uses cosine distance)
                    similarity_score = 1 - distance

                    if similarity_score >= score_threshold:
                        retrieved_docs.append({
                            'id': doc_id,
                            'content': document,
                            'metadata': metadata,
                            'similarity_score': similarity_score,
                            'distance': distance,
                            'rank': i + 1
                        })

                print(f"Retrieved {len(retrieved_docs)} documents (after filtering)")
            else:
                print("No documents found")

            return retrieved_docs

        except Exception as e:
            print(f"Error during retrieval: {e}")
            return []

rag_retriever=RAGRetriever(vectorstore,embedding_manager)



In [18]:
rag_retriever

In [19]:
rag_retriever.retrieve("What is Thermal Imaging")

# all-MiniLM-L6-v2
"""
  NO — the query does NOT need to contain all the words stored in ChromaDB.
  ChromaDB uses semantic search, not keyword matching. That means: It compares meaning, not exact words.

  Ex:
    Stored document text in ChromaDB: "Retrieval-Augmented Generation combines information retrieval with large language models."
    My query should be: "What is the architecture of RAG?"

    How does the system “know the meaning” of a query?
      - The “meaning” comes from embeddings created by a trained neural network model.
      - We used `all-MiniLM-L6-v2` model in "EmbeddingManager" class

  What really happens when you ask a query:
    - Converted my query into an embedding vector
    - Compared with stored document embeddings
    - Documents with similar meaning are returned
"""

Retrieving documents for query: 'What is Thermal Imaging'
Top K: 5, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Generated embeddings with shape: (1, 384)
Retrieved 5 documents (after filtering)


'\n  NO — the query does NOT need to contain all the words stored in ChromaDB.\n  ChromaDB uses semantic search, not keyword matching. That means: It compares meaning, not exact words.\n\n  Ex:\n    Stored document text in ChromaDB: "Retrieval-Augmented Generation combines information retrieval with large language models."\n    My query should be: "What is the architecture of RAG?"\n\n    How does the system “know the meaning” of a query?\n      - The “meaning” comes from embeddings created by a trained neural network model.\n      - We used `all-MiniLM-L6-v2` model in "EmbeddingManager" class\n\n  What really happens when you ask a query:\n    - Converted my query into an embedding vector\n    - Compared with stored document embeddings\n    - Documents with similar meaning are returned\n'

### RAG Pipeline- VectorDB To LLM Output Generation

In [28]:
!pip install langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 5.7 MB/s eta 0:00:00


In [29]:
GROQ_API_KEY="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"

In [30]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq


load_dotenv()

# 01). Initialize the Groq LLM
# Groq is a company known for its ultra-fast AI inference platform
# groq_api_key = os.getenv("GROQ_API_KEY")
# # Initialize Groq LLM (if need to set GROQ_API_KEY environment variable)
# try:
#     groq_llm = GroqLLM(api_key=os.getenv("GROQ_API_KEY"))
#     print("Groq LLM initialized successfully!")
# except ValueError as e:
#     print(f"Warning: {e}")
#     print("Please set your GROQ_API_KEY environment variable to use the LLM.")
#     groq_llm = None
groq_api_key = GROQ_API_KEY
if not groq_api_key:
    raise ValueError("GROQ_API_KEY not found in environment variables")


llm = ChatGroq(
    groq_api_key=groq_api_key,
    model_name="llama-3.1-8b-instant", # gemma2-9b-it
    temperature=0.1,
    max_tokens=1024,
)

# 02). Simple RAG function: retrive context + generate response
def rag_simple(query, retriever, llm, top_k=3):

  #retriever the context
  results = retriever.retrieve(query, top_k=top_k)
  context = "\n\n".join([doc['content'] for doc in results]) if results else ""
  if not context:
    return "No relevant context found to answer the question."

  #generate the answer using Groq LLM
  prompt=f""" Use the following context to answer the question concisely.
      Context:{context}
      Question:{query}
      Answer:
   """

  response = llm.invoke([prompt.format(context=context, query=query)])
  return response.content

In [31]:
answer=rag_simple("What is Thermal Imaging?", rag_retriever,llm)
print(answer)

Retrieving documents for query: 'What is Thermal Imaging?'
Top K: 3, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Generated embeddings with shape: (1, 384)
Retrieved 3 documents (after filtering)
Thermal imaging is a technology that captures images based on the temperature differences in a scene, allowing for the detection of heat signatures and anomalies.


### Enhanced RAG Pipeline Features

In [32]:
# --- Enhanced RAG Pipeline Features ---
def rag_advanced(query, retriever, llm, top_k=5, min_score=0.2, return_context=False):
    """
    RAG pipeline with extra features:
    - Returns answer, sources, confidence score, and optionally full context.
    """
    results = retriever.retrieve(query, top_k=top_k, score_threshold=min_score)
    if not results:
        return {'answer': 'No relevant context found.', 'sources': [], 'confidence': 0.0, 'context': ''}

    # Prepare context and sources
    context = "\n\n".join([doc['content'] for doc in results])
    sources = [{
        'source': doc['metadata'].get('source_file', doc['metadata'].get('source', 'unknown')),
        'page': doc['metadata'].get('page', 'unknown'),
        'score': doc['similarity_score'],
        'preview': doc['content'][:300] + '...'
    } for doc in results]
    confidence = max([doc['similarity_score'] for doc in results])

    # Generate answer
    prompt = f"""Use the following context to answer the question concisely.\nContext:\n{context}\n\nQuestion: {query}\n\nAnswer:"""
    response = llm.invoke([prompt.format(context=context, query=query)])

    output = {
        'answer': response.content,
        'sources': sources,
        'confidence': confidence
    }
    if return_context:
        output['context'] = context
    return output

# Example usage:
result = rag_advanced("What is Thermal Imaging?", rag_retriever, llm, top_k=3, min_score=0.1, return_context=True)
print("Answer:", result['answer'])
print("Sources:", result['sources'])
print("Confidence:", result['confidence'])
print("Context Preview:", result['context'][:300])

Retrieving documents for query: 'What is Thermal Imaging?'
Top K: 3, Score threshold: 0.1
Generating embeddings for 1 texts...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Generated embeddings with shape: (1, 384)
Retrieved 3 documents (after filtering)
Answer: Thermal imaging is the process of capturing and displaying thermal radiation as an image, typically using a thermal imaging camera. It involves detecting temperature differences in a scene, allowing for the visualization of heat patterns and anomalies.
Sources: [{'source': 'Thermal-imaging-cameras-testing-solar-panels.pdf', 'page': 1, 'score': 0.2776018977165222, 'preview': 'contrast in high dynamic range scenes, \nand the thermal image no longer needs to \nbe adjusted manually. A thermal imaging \ncamera that has DDE is therefore well suited \nfor fast and accurate solar panel inspections.\nUseful features \nAnother useful feature for a thermal imaging \ncamera is the tagging...'}, {'source': 'Thermal-imaging-cameras-testing-solar-panels.pdf', 'page': 1, 'score': 0.2776018977165222, 'preview': 'contrast in high dynamic range scenes, \nand the thermal image no longer needs to \nbe adjusted manuall

In [34]:
# --- Advanced RAG Pipeline: Streaming, Citations, History, Summarization ---
from typing import List, Dict, Any
import time

class AdvancedRAGPipeline:
    def __init__(self, retriever, llm):
        self.retriever = retriever
        self.llm = llm
        self.history = []  # Store query history

    def query(self, question: str, top_k: int = 5, min_score: float = 0.2, stream: bool = False, summarize: bool = False) -> Dict[str, Any]:
        # Retrieve relevant documents
        results = self.retriever.retrieve(question, top_k=top_k, score_threshold=min_score)
        if not results:
            answer = "No relevant context found."
            sources = []
            context = ""
        else:
            context = "\n\n".join([doc['content'] for doc in results])
            sources = [{
                'source': doc['metadata'].get('source_file', doc['metadata'].get('source', 'unknown')),
                'page': doc['metadata'].get('page', 'unknown'),
                'score': doc['similarity_score'],
                'preview': doc['content'][:120] + '...'
            } for doc in results]
            # Streaming answer simulation
            prompt = f"""Use the following context to answer the question concisely.\nContext:\n{context}\n\nQuestion: {question}\n\nAnswer:"""
            if stream:
                print("Streaming answer:")
                for i in range(0, len(prompt), 80):
                    print(prompt[i:i+80], end='', flush=True)
                    time.sleep(0.05)
                print()
            response = self.llm.invoke([prompt.format(context=context, question=question)])
            answer = response.content

        # Add citations to answer
        citations = [f"[{i+1}] {src['source']} (page {src['page']})" for i, src in enumerate(sources)]
        answer_with_citations = answer + "\n\nCitations:\n" + "\n".join(citations) if citations else answer

        # Optionally summarize answer
        summary = None
        if summarize and answer:
            summary_prompt = f"Summarize the following answer in 2 sentences:\n{answer}"
            summary_resp = self.llm.invoke([summary_prompt])
            summary = summary_resp.content

        # Store query history
        self.history.append({
            'question': question,
            'answer': answer,
            'sources': sources,
            'summary': summary
        })

        return {
            'question': question,
            'answer': answer_with_citations,
            'sources': sources,
            'summary': summary,
            'history': self.history
        }

# Example usage:
adv_rag = AdvancedRAGPipeline(rag_retriever, llm)
result = adv_rag.query("What is Thermal Imaging?", top_k=3, min_score=0.1, stream=True, summarize=True)
print("\nFinal Answer:", result['answer'])
print("Summary:", result['summary'])
print("History:", result['history'][-1])

Retrieving documents for query: 'What is Thermal Imaging?'
Top K: 3, Score threshold: 0.1
Generating embeddings for 1 texts...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Generated embeddings with shape: (1, 384)
Retrieved 3 documents (after filtering)
Streaming answer:
Use the following context to answer the question concisely.
Context:
contrast in high dynamic range scenes, 
and the thermal image no longer needs to 
be adjusted manually. A thermal imaging 
camera that has DDE is therefore well suited 
for fast and accurate solar panel inspections.
Useful features 
Another useful feature for a thermal imaging 
camera is the tagging of thermal images 
with GPS data. This helps to localize faulty 
modules easily in large areas, e.g., in solar 
farms, and also to relate the thermal images 
to the equipment, e.g., in reports.
The thermal imaging camera should have a 
built-in digital camera so that the associated 
visual image (digital photo) can be saved 
with the related thermal image. A so-called 
fusion mode, allowing the thermal and 
visual images to be superimposed, is also 
useful. Voice and text comments that can be 
saved in the camera along with 

In [36]:
from langchain_groq import ChatGroq
from langchain_core.prompts import PromptTemplate
# from langchain.schema import HumanMessage, SystemMessage
from langchain_core.documents import Document
from langchain_core.messages import HumanMessage, AIMessage

In [37]:
class GroqLLM:
    def __init__(self, model_name: str = "gemma2-9b-it", api_key: str =None):
        """
        Initialize Groq LLM

        Args:
            model_name: Groq model name (qwen2-72b-instruct, llama3-70b-8192, etc.)
            api_key: Groq API key (or set GROQ_API_KEY environment variable)
        """
        self.model_name = model_name
        self.api_key = api_key or os.environ.get("GROQ_API_KEY")

        if not self.api_key:
            raise ValueError("Groq API key is required. Set GROQ_API_KEY environment variable or pass api_key parameter.")

        self.llm = ChatGroq(
            groq_api_key=self.api_key,
            model_name=self.model_name,
            temperature=0.1,
            max_tokens=1024
        )

        print(f"Initialized Groq LLM with model: {self.model_name}")

    def generate_response(self, query: str, context: str, max_length: int = 500) -> str:
        """
        Generate response using retrieved context

        Args:
            query: User question
            context: Retrieved document context
            max_length: Maximum response length

        Returns:
            Generated response string
        """

        # Create prompt template
        prompt_template = PromptTemplate(
            input_variables=["context", "question"],
            template="""You are a helpful AI assistant. Use the following context to answer the question accurately and concisely.

Context:
{context}

Question: {question}

Answer: Provide a clear and informative answer based on the context above. If the context doesn't contain enough information to answer the question, say so."""
        )

        # Format the prompt
        formatted_prompt = prompt_template.format(context=context, question=query)

        try:
            # Generate response
            messages = [HumanMessage(content=formatted_prompt)]
            response = self.llm.invoke(messages)
            return response.content

        except Exception as e:
            return f"Error generating response: {str(e)}"

    def generate_response_simple(self, query: str, context: str) -> str:
        """
        Simple response generation without complex prompting

        Args:
            query: User question
            context: Retrieved context

        Returns:
            Generated response
        """
        simple_prompt = f"""Based on this context: {context}

Question: {query}

Answer:"""

        try:
            messages = [HumanMessage(content=simple_prompt)]
            response = self.llm.invoke(messages)
            return response.content
        except Exception as e:
            return f"Error: {str(e)}"

